# Combined SCR C02 scGen Integration

In [ ]:
# load packages
import sys
import scanpy as sc
import scgen
import os
import sys

In [ ]:
# set up working directory
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"

In [ ]:
# set up figures directory
sc.settings.figdir = os.path.join(work_dir, "figures", "combined", "integration", "by_subproject/")
sc.set_figure_params(dpi = 600, dpi_save=600)

### Cell label == "Annotation_ 2.0"

In [ ]:
# import data
adata_scgen = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_annotated_2.0_TCR_HVG_22-02-24.h5ad"))

In [ ]:
# create subproject column
adata_scgen.obs['subproject'] = adata_scgen.obs['project'].astype(str) + "_P" + adata_scgen.obs['patient'].astype(str) + "_" + adata_scgen.obs['timepoint'].astype(str)
adata_scgen.obs["subproject"]=adata_scgen.obs["subproject"].astype("category")

In [ ]:
# define integration vars
label_key = "Annotation_2.0"
batch_key = "subproject"

In [ ]:
adata_scgen.obs["Annotation_2.0"] = adata_scgen.obs["Annotation_2.0"].tolist()
sc.pp.neighbors(adata_scgen)
sc.tl.umap(adata_scgen)

In [ ]:
# prepare object
scgen.SCGEN.setup_anndata(adata_scgen, batch_key=batch_key, labels_key=label_key)


In [ ]:
# create the model
model_scgen = scgen.SCGEN(adata_scgen)

In [ ]:
# train the model
model_scgen.train()

In [ ]:
# batch removal
adata_scgen = model_scgen.batch_removal()
adata_scgen

In [ ]:
sc.pp.neighbors(adata_scgen, use_rep="corrected_latent")
sc.tl.umap(adata_scgen)

In [ ]:
sc.pl.umap(adata_scgen, color=[label_key, batch_key], wspace=0.1)

In [ ]:
# save plots
sc.pl.umap(adata_scgen, color="Annotation_1.0", show=False, save="Combined_scgen_integrated-subproject_anno2.0_annotation1.0_umap.png")
sc.pl.umap(adata_scgen, color="Annotation_2.0", show=False, save="Combined_scgen_integrated-subproject_anno2.0_annotation2.0_umap.png")
sc.pl.umap(adata_scgen, color="sample", show=False, save="Combined_scgen_integrated-subproject_sample_id_umap.png")
sc.pl.umap(adata_scgen, color="subproject", show=False, save="Combined_scgen_integrated-subproject_subproject_umap.png")

In [ ]:
# save adata object
adata_scgen.write(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_annotated_2.0_TCR_scGen_anno2.0_integrated-subproject_01-03-24.h5ad"))